In [21]:
!pip install pyspellchecker
!pip install PyPDF2
!pip install fpdf


In [22]:
from PyPDF2 import PdfReader  # extract text from PDFs
import re  # handle text processing with regular expressions
import pandas as pd  # data manipulation and storage
from collections import Counter  # count word occurrences
from spellchecker import SpellChecker  # detect misspelled or non-English words
from fpdf import FPDF  # create PDF reports
import matplotlib.pyplot as plt  # generate visualizations


In [23]:
from PyPDF2 import PdfReader
import re
import random

PDF_FILE = "/content/Harry_Potter_(www.ztcprep.com).pdf"
OUTPUT_FILE1 = "file1.txt"
OUTPUT_FILE2 = "file2.txt"

BIRTH_MONTH, BIRTH_DATE, BIRTH_YEAR = 7, 23, 2002
BOOK_ID = 7
START_PAGE1 = BIRTH_DATE+1  # pages 24-33
START_PAGE2 = 102 +1  # pages 103-112

def extract_pages(pdf_path, start_page, num_pages=10):
    """Extracts text from a specified page range in a PDF file."""
    reader = PdfReader(pdf_path)
    total_pages = len(reader.pages)

    start_index = max(0, start_page - 1)  # Convert to zero-based index
    end_index = min(start_index + num_pages, total_pages)  # Ensure within bounds

    extracted_text = []
    for i in range(start_index, end_index):
        text = reader.pages[i].extract_text()
        if text:
            extracted_text.append(text)

    return "\n".join(extracted_text)

def paraphrase_text(text):
    """Simple paraphrasing by replacing words with synonyms and restructuring sentences."""
    if not text.strip():
        return "No valid text extracted."

    # Synonyms mapping (Basic replacement method)
    synonyms = {
        "said": "stated", "looked": "glanced", "ran": "sprinted", "saw": "noticed",
        "house": "home", "boy": "young man", "girl": "young woman", "wizard": "sorcerer"
    }

    words = text.split()
    paraphrased_words = [synonyms.get(word, word) for word in words]

    paraphrased_text = " ".join(paraphrased_words)

    # Random sentence restructuring (Basic shuffle)
    sentences = re.split(r'(\.|\?|!)', paraphrased_text)  # Split into sentences
    sentences = ["".join(x) for x in zip(sentences[0::2], sentences[1::2])]  # Merge punctuation back
    random.shuffle(sentences)  # Shuffle to change order

    return " ".join(sentences)

try:
    extracted_text1 = extract_pages(PDF_FILE, START_PAGE1)
    extracted_text2 = extract_pages(PDF_FILE, START_PAGE2)

    # Apply paraphrasing
    paraphrased_text1 = paraphrase_text(extracted_text1)
    paraphrased_text2 = paraphrase_text(extracted_text2)

    with open(OUTPUT_FILE1, "w", encoding="utf-8") as f1, open(OUTPUT_FILE2, "w", encoding="utf-8") as f2:
        f1.write(paraphrased_text1)
        f2.write(paraphrased_text2)

    print(f"Text extraction and paraphrasing complete: {OUTPUT_FILE1}, {OUTPUT_FILE2}")

except Exception as e:
    print(f"Error: {e}")


Text extraction and paraphrasing complete: file1.txt, file2.txt


**Q1**:  Write Python code and use MapReduct to count occurrences of each word in the first text file (file.txt). How many times each word is repeated?

In [26]:
import re
import csv
from collections import Counter

TEXT_FILE = "/content/file1.txt"
OUTPUT_CSV = "word_count.txt"

def extract_words(text):
    """Extracts words from text and converts them to lowercase."""
    return re.findall(r'\b\w+\b', text.lower())

# Read text from file
try:
    with open(TEXT_FILE, "r", encoding="utf-8") as file:
        content = file.read()
except FileNotFoundError:
    print(f"Error: The file {TEXT_FILE} was not found.")
    exit()

# Count word occurrences
word_freq = Counter(extract_words(content))

# Save results to CSV
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Word", "Count"])
    writer.writerows(word_freq.most_common())  # Sorts by frequency automatically

# Display output in a structured format
print("\nWord Frequency Analysis from file1.txt:")
for word, count in word_freq.most_common():
    print(f"{word}: {count}")



Word Frequency Analysis from file1.txt:
the: 81
and: 49
he: 44
a: 40
it: 30
of: 28
to: 26
s: 25
dumbledore: 23
i: 23
harry: 23
his: 23
t: 19
on: 18
professor: 18
that: 17
as: 17
in: 17
mcgonagall: 16
you: 15
was: 15
hagrid: 14
stated: 13
all: 13
e: 12
him: 12
potter: 12
out: 11
be: 11
ztcprep: 10
www: 10
can: 10
but: 10
y: 9
at: 9
over: 9
her: 9
com: 9
up: 8
had: 8
g: 8
w: 7
k: 7
rowling: 7
very: 7
p: 7
philosophers: 7
stone: 7
j: 7
couldn: 7
they: 7
could: 6
front: 6
with: 6
not: 6
here: 6
ve: 6
man: 6
this: 6
re: 6
blankets: 5
know: 5
back: 5
street: 5
ll: 5
have: 5
their: 5
letter: 5
little: 5
f: 5
people: 5
for: 5
them: 5
sir: 5
motorcycle: 5
would: 5
how: 5
no: 5
see: 4
bundle: 4
took: 4
es: 4
into: 4
get: 4
will: 4
who: 4
glanced: 4
famous: 4
like: 4
were: 4
around: 4
she: 4
saying: 4
kill: 4
young: 4
just: 3
number: 3
four: 3
put: 3
turned: 3
down: 3
privet: 3
drive: 3
suddenly: 3
muggles: 3
is: 3
dursleys: 3
door: 3
almost: 3
been: 3
night: 3
when: 3
live: 3
we: 3
ou: 3
d: 3
b

**Q2**: From the second text file (file2.txt), write Python code and use MapReduct to count how many times non-English words (names, places, spells etc.) were used. List those words and how many times each was repeated.



In [27]:
import re
import csv
from collections import Counter
from spellchecker import SpellChecker

FILE_PATH = "/content/file2.txt"
OUTPUT_FILE = "non-english-words.txt"

# Initialize spell checker
spell_checker = SpellChecker()

def get_words(text):
    """Extract words from text and convert them to lowercase."""
    return re.findall(r'\b\w+\b', text.lower())

# Read text from file
try:
    with open(FILE_PATH, "r", encoding="utf-8") as file:
        content = file.read()
except FileNotFoundError:
    print(f"Error: The file {FILE_PATH} was not found.")
    exit()

# Extract words and filter non-English words
words = get_words(content)
non_english = [word for word in words if word not in spell_checker]

# Count occurrences
word_counts = Counter(non_english)

# Save results to CSV
with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Non-English Word", "Count"])
    writer.writerows(word_counts.most_common())  # Sorts by frequency automatically

# Display output
print("\nIdentified Non-English Words from file2.txt:")
for word, count in word_counts.most_common():
    print(f"{word}: {count}")



Identified Non-English Words from file2.txt:
hagrid: 27
ter: 22
yeh: 13
www: 10
ztcprep: 10
gringotts: 7
didn: 6
ernon: 5
ll: 5
stuf: 3
ap: 3
knuts: 2
izards: 2
albus: 2
eah: 2
ve: 2
gettin: 2
wasn: 1
ou: 1
mentionin: 1
ophet: 1
guardin: 1
speakin: 1
diagon: 1
65: 1
payin: 1
deliverin: 1
70: 1
dumbled: 1
ying: 1
tryin: 1
insul: 1
meself: 1
dif: 1
ficult: 1
mm: 1
everythin: 1
goin: 1
ther: 1
muggles: 1
cept: 1
72: 1
67: 1
askin: 1
pposed: 1
wouldn: 1
teh: 1
68: 1
aren: 1
messin: 1
fetchin: 1
66: 1
shouldn: 1
69: 1
71: 1
